# Immigration Data ETL
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

## Step 2 Explore and assess the data

### Methodical Approach
Lorem Ipsum

Transformation actions will be noted in brackets e.g. (TFA-1) for the first action and so on. This is to keep an overview of what needs to be implemented in the etl.py.

### Preparations

#### Imports and Installs

In [1]:
# Do all imports and installs here
import pandas as pd
import numpy as np
import logging
import sys
from datetime import datetime
from os.path import getsize
from nb_helpers import summarize_data, get_sas_definitions, read_sas_in_chunks, read_csv_print

# Logging
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s %(levelname)s \t %(message)s ',
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout,
)
log = logging.getLogger('log')

# Improve view
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


### Reading data files into Spark Dataframes
The next cell contains the file locations for each dataset

In [2]:
# File locations
dem_file = "us-cities-demographics.csv"
sas_file = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
air_file = 'airport-codes_csv.csv'
# If working on a sample of the large immigration dataset, set your sample size here:
sample_size = 0.05

Then we setup the Spark Session and read all 3 files into a dataframe

In [3]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

imm_df = spark.read.format('com.github.saurfang.sas.spark').load(sas_file)
air_df = spark.read.csv(air_file, header=True)
dem_df = spark.read.csv(dem_file, header=True)
df_list = {'imm_df': imm_df, 'air_df': air_df, 'dem_df': dem_df}
imm_full = imm_df
imm_df = imm_full.sample(sample_size)

### Explore and Assess

#### Display schemas created automatically by Spark

In [4]:
imm_df.printSchema()
air_df.printSchema()
dem_df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

**Note** (TFA-1) Demographics data file was imported as a one-column file due to ';' being used as delimiter. We read the file again:

In [5]:
dem_df = spark.read.csv(dem_file, header=True, sep=';')
dem_df.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: string (nullable = true)
 |-- Male Population: string (nullable = true)
 |-- Female Population: string (nullable = true)
 |-- Total Population: string (nullable = true)
 |-- Number of Veterans: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Average Household Size: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)



**Note**: (TFA-2) For easier handling we strip columns of spaces, apply lower case and replace inner spaces and '-' with underscore '_'

In [9]:
def trim_col_headers(df):
    for col in df.columns:
        changed_col = col.strip().lower().replace(' ', '_').replace('-', '_')
        df = df.withColumnRenamed(col, changed_col)
    return df

result_df = trim_col_headers(dem_df)
result_df.columns

['city',
 'state',
 'median_age',
 'male_population',
 'female_population',
 'total_population',
 'number_of_veterans',
 'foreign_born',
 'average_household_size',
 'state_code',
 'race',
 'count']

### Missing values

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, LongType, TimestampType, DateType
from pyspark.sql.functions import pandas_udf

In [11]:
imm_df.count()

154386

In [34]:
re

AttributeError: 'NoneType' object has no attribute 'printSchema'

### Date Transformations

In [43]:
def sasdate_to_date(df=None, column_list=[]):
    #epoch_start = spark.createDataFrame([('1960-01-01',)], ['dt'])
    df = df.withColumn('epoch_start', F.lit("01-01-1960 00:00:00"))
    df = df.withColumn('epoch_start', F.to_date(F.col('epoch_start'), "dd-M-yyyy"))
    for column in column_list:
        df = df.withColumn(column, df.arrdate.cast('int'))
        df = df.withColumn(column, F.expr("""date_add(epoch_start, df.column)"""))
    return df

result_df = sasdate_to_date(imm_df, ['arrdate', 'depdate'])
result_df.select('arrdate').show()

                 #F.col('epoch_start'), F.date_add('epoch_start', F.col('arrdate'))).show()



#work_df = work_df.withColumn('epoch_start_new', to_date(work_df.epoch_start, 'yyyy-MM-dd'))
#work_df = work_df.withColumn('new_arr', date_add('epoch-start', 'arrdate'))
#print(type(result_df), result_df.head(), result_df.printSchema())

AnalysisException: "cannot resolve '`column`' given input columns: [arrdate, depdate, fltno, i94addr, matflag, entdepu, biryear, gender, i94bir, admnum, i94res, airline, i94cit, i94visa, cicid, dtadfile, entdepa, insnum, visatype, occup, i94mode, i94yr, dtaddto, epoch_start, visapost, count, entdepd, i94mon, i94port]; line 1 pos 22;\n'Project [cicid#0, i94yr#1, i94mon#2, i94cit#3, i94res#4, i94port#5, 'date_add(epoch_start#2792, 'column) AS arrdate#2852, i94mode#7, i94addr#8, depdate#9, i94bir#10, i94visa#11, count#12, dtadfile#13, visapost#14, occup#15, entdepa#16, entdepd#17, entdepu#18, matflag#19, biryear#20, dtaddto#21, gender#22, insnum#23, ... 5 more fields]\n+- Project [cicid#0, i94yr#1, i94mon#2, i94cit#3, i94res#4, i94port#5, cast(arrdate#6 as int) AS arrdate#2822, i94mode#7, i94addr#8, depdate#9, i94bir#10, i94visa#11, count#12, dtadfile#13, visapost#14, occup#15, entdepa#16, entdepd#17, entdepu#18, matflag#19, biryear#20, dtaddto#21, gender#22, insnum#23, ... 5 more fields]

In [170]:
work_df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable =

In [110]:
df_list = {'imm_df': imm_df, 'air_df': air_df, 'dem_df': dem_df}
for name, df in df_list.items():
    pass

work_df.select([count(when(isnan('entdepd') | col('entdepd').isNull(), True))]).show()

+--------------------------------------------------------------------+
|count(CASE WHEN (isnan(entdepd) OR (entdepd IS NULL)) THEN true END)|
+--------------------------------------------------------------------+
|                                                              138429|
+--------------------------------------------------------------------+



### Exporting the data to Parquet

In [ ]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





# Step 1 Section I - Scoping and Data Gathering
**Task: Scope the Project and Gather Data**

*Identify and gather the data you'll be using for your project (at least two sources and more than 1 million rows). See Project Resources for ideas of what data you can use.*

*Explain what end use cases you'd like to prepare the data for (e.g., analytics table, app back-end, source-of-truth database, etc.)*


## Step 1a - General Scope and Data Gathering Description
The Udacity provided datasets for the Capstone Project include:
* I94 Immigration data from 2016 provided by U.S. Customs and Border Protection agency
* World Temperature Data
* U.S. cities demographic data
* An airport code table

Each dataset has been collected at least once for assessment. The findings are included in the following chapters of this notebook, even if the dataset is not used in Step 2.

Regarding the scope itself the following findings are relevant:
* **I94 Immigration data** is considered **in scope** regarding the following analytical tasks:
    * Develop a scalable automated extraction procedure using Spark Data Lake
    * Load and Transform the data into fact and dimension tables
    * Develop Airflow routines to manage the process
* **Airport Codes** are considered **in scope** and will be used
    * to enrich the immigration dataset with complete and updated values
* **World Temperature data** is considered **out of scope** since no analytics questions for this dataset in conjunction with immigration data could be identified _and_ the datasets' time periods do not overlap
* **Demographic data** is considered

**Approach to describe and gather data**

Descriptions for each dataset will be given in the sections below. Each description shall include:
1. A first read of the dataset using Python and Pandas default methods
1. "First Impression" notes about the extracted data
1. Analysis of dataset documentation, enclosed data dictionaries, etc.
1. Findings about Data Meaning, Quality, possible relationsships and definitions for
    1. Numeric columns (including missing values, uniqueness and descriptive statistics)
    1. Non-numeric columns

## Step 1 Section II - I94 Dataset of U.S. Customs and Border Protection department

### A - I94 Immigration Dataset Description
The dataset provided contains immigration data provided by US immigration authorities. Data is collected via form **I94** and contains data about people travelling from and to the US on people who are either **non United States citizens** or **lawful permanent residents** in the US.

    “Form I-94, the Arrival-Departure Record Card, is a form used by the U.S. Customs and Border Protection (CBP) intended to keep track of the arrival and departure to/from the United States of people who are not United States citizens or lawful permanent residents (with the exception of those who are entering using the Visa Waiver Program or Compact of Free Association, using Border Crossing Cards, re-entering via automatic visa revalidation, or entering temporarily as crew members)” (https://en.wikipedia.org/wiki/Form_I-94)

An overview of this dataset is also outlined [here] (https://travel.trade.gov/research/programs/i94/description.asp)

Data files and formats:
- Data files are stored in SAS (proprietary?) sas7bdat format
- Per year a folder exists
- Per month a file exists (~500 GB)

Description file:
- A description file for the fields was included, named *I94_SAS_Labels_Descriptions.SAS*
- The file contains field descriptions for each column
- And it contains value constraints for some columns, namely: *i94cnty, i94port, i94mode, i94addr*

### B - I94 Immigration Data Data Gathering and first read

As Pandas has a method to import SAS data we will be using this mechanism. The following code will read a defined number of lines only due to performance reasons.

Reading the whole SAS file in the workspace resulted in long wait times for no obvious reason. Downloading and reading the file locally (4-Cores, 8GB laptop) took only 2-3 minutes.

In [2]:
# Read in the data using a wrapper for the read_sas() method
# Configuration
sas_file =  '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
sas_file_format = 'sas7bdat'
max_lines=100000     # Set the desired line number here
for_lines=50000     # Set the desired lines for each cycle here

sas_df = read_sas_in_chunks(sas_file, sas_file_format, max_lines, for_lines)


'START reading SAS file ../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat, total filesize is: 450 Mb'

'Importing lines from 1 to 50000 of total 100000 lines'

'Importing lines from 50001 to 100000 of total 100000 lines'

'STOP reading SAS files'

'First lines of data and data types:'

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,37.0,2.0,1.0,NaN,NaN,NaN,T,NaN,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811,SEO,NaN,G,NaN,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,1.0,20160401,NaN,NaN,T,O,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,float64,float64,float64,float64,float64,object,float64,float64,object,float64,float64,float64,float64,object,object,object,object,object,object,object,float64,object,object,object,object,float64,object,object


'DONE reading SAS data in chunks, time elapsed is    3.73 seconds'

**Summary on first read of data:**

| **Column Name** | **Notes, comments and findings:**                  | **Recommended action items** |
|------------|--------------------------------------------------|----------------------------|
| all | (N) We are not importing everything here, since the files amount to about 6GB in total | |
| |  (N) Using "chunksize" parameter and then breaking from the loop, so that we have handy 2.000 lines ||
| all | In total 28 columns exist, 15 columns contain strings (object type) and 13 contain numbers (float64 type) | |
| all | At first sight one can already spot unfamiliar date columns (arrdate, depdate, etc.) with various datatypes | |
| all | Several rows have missing values | |
| all | Some columns contain obviously integer values but float64 was assigned | |
| all | Some categorical columns seem to exist | |


### C - Documentation and data description analysis


**Documentation sources**

A link was provided to the data source at the Visitor Arrivals Program: [LINK](https://travel.trade.gov/research/reports/i94/historical/2016.html)

Following related pages were also analyzed:
* Approved listing of [countries by world region](https://travel.trade.gov/research/programs/i94/1999-2020%20Region%20Dictionary.xlsx), stored as Excel file
** Recommendation: use as source to validate data
* A [Q&A section](https://travel.trade.gov/research/programs/ifs/qamythbuster.asp) which contains indications for completeness and accuracy of data
** Those should be considered before making assumptions or draw conclusions from the data
* Detailed descriptions about data collection [methodology](https://travel.trade.gov/research/programs/ifs/description.asp)

**Parsing the description file / data dictionary**

The workspace contains a field description file for the dataset named `I94_SAS_Labels_Descriptions.SAS`

The file seems pretty well structured, so I wrote a quick parser to automatically check the description file (see [SAS-Description-Parser](https://r766466c839826xjupyterlnnfq3jud.udacity-student-workspaces.com/lab/tree/SAS-Description-Parser.ipynb) for further details).

**Definitions**

| **Variable name** | **Data Type** | **Description** |
|---------------|---------------|---------------|
| i94yr | float64 | 4 digit year |
| i94mon | float64 | Numeric month |
| i94cit | float64 | This format shows all the valid and invalid codes for processing |
| i94res | float64 | This format shows all the valid and invalid codes for processing |
| i94port | object | This format shows all the valid and invalid codes for processing |
| arrdate | float64 | is the Arrival Date in the USA. It is a SAS date numeric field that apermament format has not been applied.  Please apply whichever date formatpermament format has not been applied.  Please apply whichever date format |
| i94mode | float64 | There are missing values as well as not reported (9) |
| i94addr | object | There is lots of invalid codes in this variable and the list belowThere is lots of invalid codes in this variable and the list below |
| depdate | float64 | is the Departure Date from the USA. It is a SAS date numeric field thata permament format has not been applied.  Please apply whichever date formata permament format has not been applied.  Please apply whichever date format |
| i94bir | float64 | Age of Respondent in Years |
| i94visa | float64 | Visa codes collapsed into three categories:1 = Business2 = Pleasure3 = Student*/ |
| count | float64 | Used for summary statistics |
| dtadfile | object | Character Date Field |
| visapost | object | Department of State where where Visa was issued |
| occup | object | Occupation that will be performed in U.S. |
| entdepa | object | Arrival Flag |
| entdepd | object | Departure Flag |
| entdepu | object | Update Flag |
| matflag | object | Match flag |
| biryear | float64 | 4 digit year of birth |
| dtaddto | object | Character Date Field |
| gender | object | Non |
| insnum | object | INS number |
| airline | object | Airline used to arrive in U.S. |
| admnum | float64 | Admission Number |
| fltno | object | Flight number of Airline used to arrive in U.S. |
| visatype | object | Class of admission legally admitting the non |

**Summary on data documentation and descriptions:**

| **Column Name** | **Notes, comments and findings:**                  | **Recommended action items** |
|------------|--------------------------------------------------|----------------------------|
| `i94cnty, i94port, i94mode, i94addr` | have value constraints (lists with allowed entry values) | Change data types, validate content (see below) | |
| `i94cnty` | contains country short codes and their corresponding state names | Change to category field |
| `i94port` | contains port/airport codes from various cities, without specific selection criteria as it seems (most of the codes are cities in the US but we also see city codes from Europe and Asia) | Change data type to string, Validate airport table using airport-codes_csv.csv| 
| `i94mode` | is a code for the way of travelling (by Air, by Sea or by Land) or unknown | Change to category field |
| `i94addr` | is a code for the state in which this immigrants temporary address is located (aka "First Intended Address") | Change to category field |


### D - Analysis of numeric columns
The Pandas describe() function creates a basic set of descriptive statistics for each numeric column in the data frame.

In [3]:
summarize_data(sas_df, ['numbers'])

'Running Data Quantifier with parameter: '

' and example threshhold is '

'The dataframe has 100000 rows and 13 columns. Godspeed!'

'Quantifying NUMERIC data types in columns:'

'cicid, i94yr, i94mon, i94cit, i94res, arrdate, i94mode, depdate, i94bir, i94visa, count, biryear, admnum'

...

"Unique values in column 'i94visa': "

'         2,          3,          1'

"Unique values in column 'count': "

'         1'

'Data Quantification Done, time elapsed is    0.37 sec'

**Summary on numeric data:**

| **Column Name** | **Notes, comments and findings:**                  | **Recommended action items** |
|------------|--------------------------------------------------|----------------------------|
| `cicid` |is unique for all 2.000 lines (check `len(sas_df['cicid'].unique())`) and appears to be the primary key for each record | Change to int, Use as primary key|
| | The following columns appear to indicate datetime related values: |
| `i94yr` |indicating the year the I94 form was filled and 'i94mon' indicating the month | Change to int |
| `arrdate` |is the immigrants arrival date | Change to datetime64 |
| `depdate` |the date of the immigrants (planned) departure | Change to datetime64 |
| `i94mode` | has already been identified as a category variable, the integers here are just codes indicating if the immigrant travelled by Land, Air or Sea (or unknown) | Change to category|
| `i94visa` | was not identified correctly by my parser it seems, it has value constraints (* 1 = Business, 2 = Pleasure,3 = Student)  | Change to category|
| `i94cit` and `i94res` | are again not numeric but indicate the immigrant's countries of citizenship ("cit") and residence (res) | Change to category, Validate date using value constraints |
|`admnum` | is the admission number | Use as key variable to connect several rows |
|`i94bir` |appears to be the immigrant's age at the time of admission (in other words it's the time delta between `i94yr`and `biryear` | Change to int|
| `biryear` | marks the immigrants birthyear | Change to int |
| `count` |is for statistical purposes according to the description | Change to int |

### E - Analysis of non-numeric columns
Measuring the number of NaN entries and unique values

In [4]:
summarize_data(sas_df, ['object'])

'Running Data Quantifier with parameter: '

' and example threshhold is '

'The dataframe has 100000 rows and 15 columns. Godspeed!'

'Quantifying NON-NUMERIC data types in columns:'

'i94port, i94addr, dtadfile, visapost, occup, entdepa, entdepd, entdepu, matflag, dtaddto, gender, insnum, airline, fltno, visatype'

...

"Unique values in column 'gender': "

'M, F'

"Unique values in column 'insnum': "

''

'Data Quantification Done, time elapsed is     1.6 sec'

**Summary on non-numeric data:**

| **Column Name** | **Notes, comments and findings:**                  | **Recommended action items** |
|------------|--------------------------------------------------|----------------------------|
| `i94prt, i94addr, dtadfile` | Value constraints exist | Change to category, Validate against list of constraints |
| `dtadfile` |is the date on which the form was entered into the database | Change to datetime64 |
| `dtaddto` |is the date the immigrant is admissioned to stay in the US | Change to datetime64 |
| `visapost, occup` | rarely filled, not fit for analysis | 
| `entdepa, entdepd,entdepu,matflag,` | Unclear description in the data dictionary | Exclude from analyis |
| `gender` | Immigrant gender | Change to category |
| `insnum` | Immigration registration number, not filled in data sample | Check if field is filled in complete dataset | |
| `visatype` | Type of issued visa | Check [online sources](https://travel.trade.gov/research/programs/i94/methodology.asp) for list of possible types |


### F - Dataset conclusion (Immigration Data)

**Summary on non-numeric data:**

| **Column Name** | **Commends and findings:**                  | **Recommended action items** |
|------------|--------------------------------------------------|----------------------------|
| <FELD> | <COMMENT, NOTE, FINDING> | <ACTION>

## Step 1 Section III - World Temperature Data

### A - World Temperature Data Description

The World Temperature Dataset contains temperatures on land by city on a global scale. A detailed discription can be found [here](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).

The source indicates that other datasets exist which summarize the data e.g. by country or major cities.

### B - Documentation Analysis

The dataset from kaggle provides a documentation on its hosting [site](https://www.kaggle.com/colinpbowen/starter-climate-change-earth-surface-e24bc90c-4)

The set consists of seven columns:
* `dt` is the temperatures measurement timestamp
* `AverageTemperature` displays the average temperature in celsius degrees
* `AgerageTemperatureUncertainty` shows possible deviations from average (95% confidence)
* `City` - name of the city, has 3.448 distinct values
* `Latitude, Longitude` - location of measurement

### C - World Temperature Data Gathering and first read

As Pandas has a method to import CSV data we will be using this mechanism.

Instead of reading just a chunk of the file we will read it in full here.

In [5]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
temperature_df = read_csv_print(fname, ',')

'START reading CSV file ../../data2/GlobalLandTemperaturesByCity.csv of (Filesize: 5.1e+02 Mb)'

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


'Done. Operation took     9.9 seconds'

**Summary on first read:**

| **Column Name** | **Commends and findings:**                  | **Recommended action items** |
|------------|--------------------------------------------------|----------------------------|
| all | Dataset contains in total over 8.5m rows | |
| `dt` | starting in 1743 | Convert to datetime64 |
| `AverageTemperature` | mind the missing values | Check for isna() |
| `AverageTemperatureUncertainty` | same as above | |
| `City, Country` | None | |
| `Latitude, Longitude` | |

### D - Analysis of numeric columns in Temperature Data

Only two numeric columns were identified:  `AverageTemperature`, `AverageTemperatureUncertainty`

In [6]:
summarize_data(temperature_df, ['numbers'])

'Running Data Quantifier with parameter: '

' and example threshhold is '

'The dataframe has 8599212 rows and 2 columns. Godspeed!'

'Quantifying NUMERIC data types in columns:'

'AverageTemperature, AverageTemperatureUncertainty'

...

'Columns with missing values:'

'AverageTemperature, AverageTemperatureUncertainty'

'Columns with less than 10 unique values:'

''

'Data Quantification Done, time elapsed is     3.4 sec'

**Summary on numeric data:**

| **Column Name** | **Notes, comments and findings:**                  | **Recommended action items** |
|------------|--------------------------------------------------|----------------------------|
| `AverageTemperature` | About 364.000 missing values | Exclude NaN from analysis |
| | 
| `AverageTemperatureUncertainty` | About 364.000 missing values, which is expected due to missing temperature data | Exclude NaN from analysis |

### E - Analysis of non-numeric columns in Temperature

Non-numeric data columns, qualitative analysis

In [7]:
summarize_data(temperature_df, ['object'])

'Running Data Quantifier with parameter: '

' and example threshhold is '

'The dataframe has 8599212 rows and 5 columns. Godspeed!'

'Quantifying NON-NUMERIC data types in columns:'

'dt, City, Country, Latitude, Longitude'

...

'Columns with missing values:'

''

'Columns with less than 10 unique values:'

''

'Data Quantification Done, time elapsed is   2e+01 sec'

In [8]:
# Convert "dt" to datetime and sort, then check latest measurement date

temperature_df[['dt']] = temperature_df[['dt']].astype('datetime64')
temperature_df = temperature_df.sort_values(by=['dt'], ascending=False)
print('The last datapoint is from the following date: {:.10}'.format(temperature_df['dt'].head(1).values[0]))

The last datapoint is from the following date: 2013-09-01


In [9]:
# Check distribution of data points per City

# Count datapoints per city
count_df = temperature_df[['City', 'dt']].copy()
count_df = count_df.groupby(by=['City']).count()
count_df = count_df.sort_values(by='dt', ascending=False)
entries = len(temperature_df)
num_of_cit = len(count_df)
print('Average datapoints per city: {:6.0f} entries'.format((entries / num_of_cit)))
print('City with most data points: {}\t\t\t{} entries'.format(count_df.head(1).index[0], count_df['dt'].head(1).values[0]))
print('City with least data points: {}\t\t\t{} entries'.format(count_df.tail(1).index[0], count_df['dt'].tail(1).values[0]))


Average datapoints per city:   2494 entries
City with most data points: Springfield			9545 entries
City with least data points: Port Moresby			1581 entries


**Summary:**

| **Column Name** | **Notes, comments and findings:**                  | **Recommended action items** |
|------------|--------------------------------------------------|----------------------------|
| `City, dt` | city names and timestamps are one possible way to **link this dataset to the I94 immigration data** | Join datasets on a combined key of both fields
| `dt` | The "freshest" data point is from September 2013 |  |
| all | There is a significant **imbalance in the number of data points**: While the average is 2.494 entries, the city with most entries has 9.545 and the city with least entries has 1.581 entries | |
| all | Also the datapoints per day are varying between about 700 and 3500 per day | |


### F - Dataset conclusion

The "World Temperature Dataset" could be linked to the I94 immigration data by a combined key of `City`and `dt`. This would allow to add temperature data as dimension table when analyzing immigration data.

However the dataset is not well suited to be analyzed in conjunction with the Immigration Data, since there is **no time period overlap**. Without temperature data from the time period of the provided immigration data no signifant findings from data analyses can be expected.

**Conclusion**: the World Temperature Dataset is ruled **out of scope**.

## Step 1 Section IV - U.S. City Demographic Data

### A - Demographic Data Description

A dataset of demographic data is provided from [opendatasoft](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/). It contains statistic about the population structure in various U.S. cities with a population greater or equal to 65,000.

The data was gathered in the US Census Bureau's 2015 American Community Survey.

### B - Demographic Data Gathering and first read

We will use Pandas standard CSV reading method and read the complete file

In [10]:
fname = "us-cities-demographics.csv"
dem_data_df = read_csv_print(fname)

'START reading CSV file us-cities-demographics.csv of (Filesize:    0.24 Mb)'

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


'Done. Operation took   0.044 seconds'

### C - Documentation Analysis

The dataset's documentation is rather brief and available  [online](https://public.opendatasoft.com/explore/embed/dataset/us-cities-demographics/table/?dataChart=eyJxdWVyaWVzIjpbeyJjb25maWciOnsiZGF0YXNldCI6InVzLWNpdGllcy1kZW1vZ3JhcGhpY3MiLCJvcHRpb25zIjp7fX0sImNoYXJ0cyI6W3siYWxpZ25Nb250aCI6dHJ1ZSwidHlwZSI6ImNvbHVtbiIsImZ1bmMiOiJBVkciLCJ5QXhpcyI6Im1lZGlhbl9hZ2UiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiIjRkY1MTVBIn1dLCJ4QXhpcyI6ImNpdHkiLCJtYXhwb2ludHMiOjUwLCJzb3J0IjoiIn1dLCJ0aW1lc2NhbGUiOiIiLCJkaXNwbGF5TGVnZW5kIjp0cnVlLCJhbGlnbk1vbnRoIjp0cnVlfQ%3D%3D). Meanings of column headings are pretty self-explanatory. No deeper analysis required.

### D - Analysis of numeric columns

The demographic data contains various numeric columns.

In [12]:
summarize_data(dem_data_df, ['numbers'])

'Running Data Quantifier with parameter: '

' and example threshhold is '

'The dataframe has 2891 rows and 8 columns. Godspeed!'

'Quantifying NUMERIC data types in columns:'

'Median Age, Male Population, Female Population, Total Population, Number of Veterans, Foreign-born, Average Household Size, Count'

...

'Columns with missing values:'

'Male Population, Female Population, Number of Veterans, Foreign-born, Average Household Size'

'Columns with less than 10 unique values:'

''

'Data Quantification Done, time elapsed is    0.11 sec'

**Summary:**

* All columns contain integer values with the exception of `Median Age` and `Average Household Size` which are floating point values
* Only a limited amounts of values are missing,the dataset seems rather complete

### E - Analysis of non-numeric columns

The demographic dataset contains some string data which will be analyzed here.

In [13]:
summarize_data(dem_data_df, ['object'])

'Running Data Quantifier with parameter: '

' and example threshhold is '

'The dataframe has 2891 rows and 4 columns. Godspeed!'

'Quantifying NON-NUMERIC data types in columns:'

'City, State, State Code, Race'

...

'Columns with less than 10 unique values:'

'Race'

"Unique values in column 'Race': "

'Hispanic or Latino, White, Asian, Black or African-American, American Indian and Alaska Native'

'Data Quantification Done, time elapsed is   0.094 sec'

**Summary:**

| **Column Name** | **Notes, comments and findings:**                  | **Recommended action items** |
|------------|--------------------------------------------------|----------------------------|
| `City` | can be used to link data to I94 immigration dataset | Apply as key |
| `State Code` | can be used to link data to I94 immigration dataset | Apply as key |
| `Race` | Only 5 different values exist indicating this column as a category | Add category/ dimension |
| `all` | No missing values | |

### F - Dataset conclusion

**Summary:**

* The demographic dataset appears to be of good quality with few missing values
* The dataset has only a limited files of 0.24 Mb
* It can be linked to the immigration dataset, for instance to research relations between demographics and travelling patterns

**Conclusion**: The dataset on U.S. city demographics is considered **in scope**

## Step 1 Section V - Airport Code Table

### A - Airport Code Table Data Description

A list of airports from [datahub.io](https://datahub.io/core/airport-codes#data) with various datapoints was provided for this project.

### B - Airport Code Table Data Gathering and first read

Again we will be using Pandas CSV Reader and import the complete file.

In [16]:
fname = 'airport-codes_csv.csv'
airport_df = read_csv_print(fname, ',')

'START reading CSV file airport-codes_csv.csv of (Filesize:     5.7 Mb)'

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


'Done. Operation took    0.28 seconds'

### C - Documentation Analysis

According to the documentation found online, the list contains the following data:
* either IATA airport codes consisting of 3 letters
* or ICAO codes consisting of 4 letters

The list contains over 50.000 entries for airports and adds information for each airport such as:
* additional codes like local codes
* location information such as city, country and geo-coordinates
* elevation of the airport above sea-level
* type of airport (is it Newark or just a helipad?)

### D - Analysis of numeric columns

`elevation_ft` is the only numeric field in the dataset

In [34]:
summarize_data(airport_df, 'numbers')

'Running Data Quantifier with parameter: '

' and example threshhold is '

'The dataframe has 55075 rows and 1 columns. Godspeed!'

'Quantifying NUMERIC data types in columns:'

'elevation_ft'

...

'Columns with missing values:'

'elevation_ft'

'Columns with less than 10 unique values:'

''

'Data Quantification Done, time elapsed is   0.072 sec'

### E - Analysis of non-numeric columns

In [37]:
summarize_data(airport_df, 'objects')

'Running Data Quantifier with parameter: '

' and example threshhold is '

'The dataframe has 55075 rows and 11 columns. Godspeed!'

'Quantifying NON-NUMERIC data types in columns:'

'ident, type, name, continent, iso_country, iso_region, municipality, gps_code, iata_code, local_code, coordinates'

...

"Unique values in column 'type': "

'heliport, small_airport, closed, seaplane_base, balloonport, medium_airport, large_airport'

"Unique values in column 'continent': "

'OC, AF, AN, EU, AS, SA'

'Data Quantification Done, time elapsed is    0.39 sec'

**Summary on non-numeric columns:**

| **Column Name** | **Notes, comments and findings:**                  | **Recommended action items** |
|------------|--------------------------------------------------|----------------------------|
| `continent, iso_country, municipality, gps_code, iata_code, local_code` | all fields would be suitable analytics dimensions, but the `*_code`columns show a high number of empty cells | Analyze possible hidden duplicates (e.g. by comparing location data) to improve data quality |
| `type, continent` | while `type` columns is completely filled, `continent` has about 50% missing values | Use column `iso_country` to update continents |
| `iata_code` | high amount of missing values but still should be tried to match to column `i94port` of the immigration dataset |
| `iso_region, municipality` | Possibly matching to `i94addr` | Remove prefix "US-" and join tables |

### F - Dataset conclusion on airport codes

**Summary:**

* The dataset offers various analytical dimensions e.g.
    * the `type` column adds a bit of detail to airports immigrants use
    * the `coordinates` column may be used to create geo-visulizations
    * `name` and `iata_codes` can be used to enrich available data points
* The dataset should be considered in scope since it is possible to link it with immigration data using one of two columns.

## Step 1 Summary

For analyzing the immigration dataset 3 possible datasets were suggested as possible analysis dimensions and data sources: world temperature data, U.S. cities demographic data and a detailed information about airports

The world temperature dataset was moved out of the project scope since it does not contain temperature data from the available immigration dataset's time period.

The airport codes and demographics data can be linked with the immigration data via location names or codes. Thus it is being considered in scope of this project.

While both datasets are rather limited in size (5.7 and 0.24 Megabytes) the immigration dataset is comparably large. For production use a scalable import tool is required.

For many columns quality characteristics were identified like suggesting possible categorical data types, fill methods for missing values and relationship key fields. Those recommendations will be implemented in Step 4 of this project.

Step 1 of this project created a more technical understanding of the data and its quality. In Step 2 some explorative analysis will be performed to learn more about required data cleaning actions.

---

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.